Universidad del valle de Guatemala  
Dpto. Ciencias de la computacion  
Inteligencia Artificial  
Alberto Suriano  

Laboratorio 4  
Andres Quinto - 18288  
Marlon Hernández - 15177

- Link del repositorio: https://github.com/AndresQuinto5/IA_LAB4.git

# Task 1 - Preguntas Teóricas

1. Explique con sus propias palabras, qué son los grafos computaciones y cuál es su importancia para el
cálculo de gradientes en aplicaciones como backpropagation
2. Detalle cuales son los componentes y pasos que conforman una red neuronal. Con esto en mente, ¿cómo
mejoraría el perceptrón que hizo en el laboratorio pasado?
3. Investigue y explique cómo se selecciona el valor K usando el método de la silueta para el algoritmo de
K-Means. Explique las fórmulas (ecuaciones) que lo componen así como las asumpciones, si hay.
4. Investigue sobre Principal Component Analysis (PCA) y responda respecto a algoritmos como K-Means:
a. ¿Cómo podría ayudarle a mejorar la calidad de sus clusters cuando se usa K-Means?